In [1]:
!pip install pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

In [3]:
(train_images,train_labels), (test_images,test_labels) = keras.datasets.mnist.load_data()

print(f"trening -> {train_images.shape}")
print(f"trening -> {test_images.shape}")

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 0s 0us/step
trening -> (60000, 28, 28)
trening -> (10000, 28, 28)


In [4]:
train_images.shape

(1000, 784)

In [5]:
test_images.shape

(1000, 784)

In [6]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

In [7]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
#użycie wywołania zwrotnego w punkcie kontrolmym
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)


model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 1.2345 - sparse_categorical_accuracy: 0.6570
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 5s 55ms/step - loss: 1.2345 - sparse_categorical_accuracy: 0.6570 - val_loss: 0.7653 - val_sparse_categorical_accuracy: 0.7730
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.4686 - sparse_categorical_accuracy: 0.8570
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 27ms/step - loss: 0.4686 - sparse_categorical_accuracy: 0.8570 - val_loss: 0.5294 - val_sparse_categorical_accuracy: 0.8440
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.3165 - sparse_categorical_accuracy: 0.9120
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.3165 - sparse_categorical_accuracy: 0.9120 - val_loss: 0.4592 - val_sparse_categorical_accuracy: 0.8630
Epoch 4/10
29/32 [=======

In [9]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [10]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3571 - sparse_categorical_accuracy: 0.1000 - 201ms/epoch - 6ms/step
Untrained model, accuracy: 10.00%


In [11]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3950 - sparse_categorical_accuracy: 0.8790 - 97ms/epoch - 3ms/step
Restored model, accuracy: 87.90%


In [22]:
#wywołanie zwrotne z opcją oddzwaniania do punktu kontrolnego
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32


In [23]:
cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [24]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [25]:
model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4825 - sparse_categorical_accuracy: 0.8750 - 241ms/epoch - 8ms/step
Restored model, accuracy: 87.50%


In [26]:
model = create_model()

model.fit(train_images,
          train_labels,
          epochs=5)

!mkdir -p saved_model
model.save('saved_model/mojmodel')

Epoch 1/5
32/32 [==============================] - 1s 10ms/step - loss: 1.2205 - sparse_categorical_accuracy: 0.6520
Epoch 2/5
32/32 [==============================] - 0s 10ms/step - loss: 0.4462 - sparse_categorical_accuracy: 0.8660
Epoch 3/5
32/32 [==============================] - 0s 9ms/step - loss: 0.3132 - sparse_categorical_accuracy: 0.9170
Epoch 4/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2241 - sparse_categorical_accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 11ms/step - loss: 0.1732 - sparse_categorical_accuracy: 0.9570


In [27]:
!ls saved_model

mojmodel


In [28]:
!ls saved_model/mojmodel

assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


In [29]:
new_model = keras.models.load_model('saved_model/mojmodel')

new_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 512)               401920    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_19 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4230 - sparse_categorical_accuracy: 0.8630 - 249ms/epoch - 8ms/step
Restored model, accuracy: 86.30%


In [31]:
model = create_model()

model.fit(train_images,
          train_labels,
          epochs=5)

model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 14ms/step - loss: 1.2037 - sparse_categorical_accuracy: 0.6570
Epoch 2/5
32/32 [==============================] - 0s 14ms/step - loss: 0.4425 - sparse_categorical_accuracy: 0.8750
Epoch 3/5
32/32 [==============================] - 1s 17ms/step - loss: 0.3075 - sparse_categorical_accuracy: 0.9150
Epoch 4/5
32/32 [==============================] - 0s 13ms/step - loss: 0.2376 - sparse_categorical_accuracy: 0.9400
Epoch 5/5
32/32 [==============================] - 0s 16ms/step - loss: 0.1710 - sparse_categorical_accuracy: 0.9590


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
hdf5_model = keras.models.load_model('my_model.h5')
hdf5_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 512)               401920    
                                                                 
 dropout_10 (Dropout)        (None, 512)               0         
                                                                 
 dense_21 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
loss, acc = hdf5_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4463 - sparse_categorical_accuracy: 0.8550 - 261ms/epoch - 8ms/step
Restored model, accuracy: 85.50%
